# TP1 - Pandas y Visualización de datos

---



P36. Consiga una matriz cuyas columnas sean los cast id y crew ids (diferenciados para no colisionar), que tenga una fila por cada película y que los valores sean la cantidad de roles para los que esa persona participa en la película. Estime cuánto ocuparía en RAM teniendo en cuenta que cada número pesa 4 bytes, y en caso de no entrar construyala como matriz dispersa.

In [6]:
import pandas as pd
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cast_original = pd.read_csv("/content/drive/MyDrive/Organización de Datos/TP1/Movie dataset/cast.csv")
crew_original = pd.read_csv("/content/drive/MyDrive/Organización de Datos/TP1/Movie dataset/crew.csv")

In [11]:
cast = cast_original[ ['id', 'cast_id'] ].dropna()
crew = crew_original[ ['id', 'crew_id'] ].dropna()

cast = cast.groupby(['cast_id', 'id'], sort=False).size().to_frame().reset_index()
cast.columns = ['cast_id', 'id', 'cant_roles']
cast["cast_id"] = cast["cast_id"].map(lambda x : "cast_" + str(x))

crew = crew.groupby(['crew_id', 'id'], sort=False).size().to_frame().reset_index()
crew.columns = ['crew_id', 'id', 'cant_roles']
crew["crew_id"] = crew["crew_id"].map(lambda x : "crew_" + str(x))

cast = cast.rename(columns={"cast_id":"worker_id"})
crew = crew.rename(columns={"crew_id":"worker_id"})

cast_crew = pd.concat([cast, crew])

In [18]:
worker_id_c = CategoricalDtype(sorted(cast_crew.worker_id.unique()), ordered=True)
id_c = CategoricalDtype(sorted(cast_crew.id.unique()), ordered=True)

filas = cast_crew.id.astype(id_c).cat.codes
columnas = cast_crew.worker_id.astype(worker_id_c).cat.codes
matriz_dispersa = csr_matrix((cast_crew["cant_roles"], (filas, columnas)), shape=(id_c.categories.size, worker_id_c.categories.size))

cast_crew_comprimido = pd.DataFrame.sparse.from_spmatrix(matriz_dispersa, index=id_c.categories, columns=worker_id_c.categories)

cast_crew_comprimido

,cast_1,cast_10,cast_100,cast_10000,cast_1000007,cast_1000061,cast_1000083,cast_1000089,cast_1000109,cast_1000145,...,crew_999882,crew_999886,crew_999909,crew_99993,crew_99994,crew_999947,crew_999948,crew_99996,crew_999980,crew_999981
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465044,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
467731,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
468343,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
468707,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
worker_ids = len(cast_crew_comprimido.columns)
pelis_ids = len(cast_crew_comprimido)
bytes = worker_ids * pelis_ids * 4
#bytes = 66208120252
tamaño_estimado = bytes / 10**9
#tamaño_estimado = 66,2 GB